# All Roads Lead to Kdb: the technical counterpart

This post is a follow-up to our previous article, [All Roads Lead to Kdb](https://www.habla.dev/blog/2023/07/31/all-roads-lead-to-pykx.html), in which we introduced the utility of the PyKX library from the perspective of Emma Monad, the CTO of a large and fictional company known as Mad Flow. Complementing this C-level perspective, this post is aimed at Pythonist programmers who are eager to delve into the technical details that were left unexplained in the aforementioned article. Eloy, Jesús, and Félix will be the fictional characters assisting us in this endeavor to understand both the goals of the traffic improvement use case undertaken by Mad Flow, as well as the technical nuances of the role played by PyKX within it. In general, and similarly to the results shown in this [post](https://kx.com/blog/accelerating-python-workflows-using-pykx/) on the [KX developer blog](https://kx.com/resources/developer-blog/) by Morrison and Crone, which we highly recommend reading, we'll observe a significant speed advantage of the resulting PyKX code over the initial Pandas implementation.

The post will be structured as follows:

* [**The Use Case**](#t2): First, we will explain the traffic improvement use case itself, including its goals, data sources, and expected results. In this section, you will find references to the necessary data sources required to execute all the code in this notebook.
* [**PyKX migration**](#t3): Next, we will demonstrate how to offload the heavy processing to kdb+/q using PyKX, while remaining within the Python environment! This PyKX-powered Pythonic version will likely be the fastest way to start taking advantage of the kdb+/q environment.
* [**pykx.q migration**](#t4): Lastly, we will elaborate on the process of transitioning the code to pure kdb+/q, while utilizing PyKX to continue relying on the Pythonic code you wish to retain. This PyKX-powered kdb+/q version is presented for those interested in adopting kdb+/q more extensively. 


## The Use Case<a class="anchor" id="t2"></a>

With the goal of predicting traffic congestion in the presence of rain, Mad Flow invested a significant amount of time in preparing and ingesting weather and traffic data into an LSTM model. This effort was in line with other [studies](https://www.mdpi.com/1424-8220/20/13/3749) that aimed to forecast traffic patterns using LSTM models based on air pollution. And similar to findings from studies conducted in cities such as [Manchester](https://pure.manchester.ac.uk/ws/portalfiles/portal/72721911/DEXA_Camera_8_pages.pdf), [Shenzhen](https://ieeexplore.ieee.org/document/8964560) and [Belgrade](https://www.safetylit.org/citations/index.php?fuseaction=citations.viewdetails&citationIds[]=citjournalarticle_716898_14), our results project a weekday traffic volume increase of 5-15% during peak hours when rain is present. 

As is usual in projects associated with smart cities, the data is notably heterogeneous, demanding substantial preparatory efforts. The next sections will describe the data sources, the cleaning and interpretation processes, as well as the model used.


### Data sources <a class="anchor" id="t21"></a>

The [Madrid City Council](https://datos.madrid.es/portal/site/egob) offers weather and traffic data, which includes both real-time and historical records. For the purpose of model training, only the historical data is necessary. These CSV-formatted datasets are organized by months spanning from the year 2018 to the present.

<div class="alert alert-block alert-info">
    <b> 🔍 You can access the different datasets required to run this notebook from the following links: 
        <ul>
            <li> <a href="https://datos.madrid.es/egob/catalogo/300352-141-meteorologicos-horarios.csv">Weather data</a>
            <li> <a href="https://datos.madrid.es/egob/catalogo/300360-1-meteorologicos-estaciones.csv">Weather stations</a>
            <li> <a href="https://datos.madrid.es/egob/catalogo/208627-115-transporte-ptomedida-historico.zip">Traffic data</a>
            <li> <a href="https://datos.madrid.es/egob/catalogo/202468-170-intensidad-trafico.csv">Traffic stations</a>
        </ul>
    </b>
</div>



**Weather data**

The following table presents a snapshot of the kind of information that we can find in the [weather dataset](https://datos.madrid.es/egob/catalogo/300352-141-meteorologicos-horarios.csv):

|    |   weather_station |   magnitude |   year |   month |   day |    H01 | V01   | ...|
|---:|------------------:|-----------:|-------:|--------:|------:|-------:|------:|:------|
|  0 |               108 |         81 |   2022 |      12 |    22 |  270   | N     | ...|
|  1 |               108 |         82 |   2022 |      12 |    22 |    9   | N     |...|
|  2 |               108 |         83 |   2022 |      12 |    22 |   94.7 | N     |...|
|  3 |               108 |         86 |   2022 |      12 |    22 | 1031   | N     |...|
|  4 |               108 |         87 |   2022 |      12 |    22 |    2   | N     |...|

Each row records several information about a particular meteorological station: 

* The 'magnitude', which indicates the type of meteorological data recorded in the respective row (e.g. 81 - wind, 82 - direction, ..., 89 - rainfall)
* In columns 'H01' and 'V01', the value for the data type indicated by the 'magnitude' column, along with its validity status, respectively.
* In columns 'year', 'month' and 'day', the time at which the measurement is recorded.

In addition to this information, we will also need the geographical coordinates of the different stations, which can be found in a separate [table](https://datos.madrid.es/egob/catalogo/300360-1-meteorologicos-estaciones.csv) provided by the Madrid City Council.


(TBD: show the station 108)
|    |   weather_station |   Longitude |   Latitude |
|---:|------------------:|------------:|-----------:|
|  0 |                 4 |    -3.71226 |    40.4239 |
|  1 |                 8 |    -3.68232 |    40.4216 |
|  2 |                16 |    -3.63924 |    40.44   |
|  3 |                18 |    -3.73184 |    40.3948 |
|  4 |                24 |    -3.74734 |    40.4194 |

**Traffic data**

Traffic sensors are located at traffic lights across the city of Madrid. The collected data gathers various measurements of the road conditions, including speed and traffic volume. All of these metrics are encapsulated in the 'load' metric, represented as a percentage that quantifies congestion. The structure of the [traffic table](https://datos.madrid.es/egob/catalogo/208627-115-transporte-ptomedida-historico.zip) is as follows: 

|    |   traffic_station | tipo_elem   |   intensidad |   ocupacion |   load |   vmed | error   |   periodo_integracion | date                |
|---:|------------------:|:------------|-------------:|------------:|-------:|-------:|:--------|----------------------:|:--------------------|
|  0 |              1001 | M30         |         3240 |           9 |      0 |     59 | N       |                     5 | 2022-12-01 00:00:00 |
|  1 |              1001 | M30         |         3240 |           9 |      0 |     59 | N       |                     5 | 2022-12-01 00:15:00 |
|  2 |              1001 | M30         |          300 |           1 |      0 |     66 | N       |                     5 | 2022-12-01 00:30:00 |


The table also contains a column for the date and another that identifies the sensor that will be used to link it with its coordinates using the next [traffic stations table](https://datos.madrid.es/egob/catalogo/202468-170-intensidad-trafico.csv):

|    | "tipo_elem"   |   "distrito" |   "id" | "cod_cent"   | "nombre"                                                          |   "utm_x" |     "utm_y" |   Longitude |   Latitude |
|---:|:--------------|-------------:|-------:|:-------------|:------------------------------------------------------------------|----------:|------------:|------------:|-----------:|
|  0 | "URB"         |            4 |   3840 | "01001"      | "Jose Ortega y Gasset E-O - Pº Castellana-Serrano"                |    441615 | 4.47577e+06 |    -3.68832 |    40.4305 |
|  1 | "URB"         |            4 |   3841 | "01002"      | "Jose Ortega y Gasset O-E - Serrano-Pº Castellana"                |    441706 | 4.47577e+06 |    -3.68726 |    40.4305 |
|  2 | "URB"         |            1 |   3842 | "01003"      | "Pº Recoletos N-S - Almirante-Prim"                               |    441319 | 4.47484e+06 |    -3.69173 |    40.4221 |



### The Cleaning  <a class="anchor" id="t22"></a>

After loading the data, the next step was to prepare it for analysis and modeling. Given the heterogeneous nature of the datasets, data cleaning will be performed separately, and the data will be consolidated into a single table at a later stage.

#### Weather

The steps followed to clean the climate table are:

1. In the weather dataset we first have the date separated into day, month and year, but in traffic all these data are together. This is why the columns have been grouped into a date column of shape: ```day-month-year```. 

2. We also transform each hour value measurement and its validity into independent rows and take the opportunity to remove invalid measurements:

|       | hour            |   value |
|------:|:----------------|--------:|
|     0 | 0 days 00:00:00 |    7.1  |
|     1 | 0 days 00:00:00 |    6.8  |
|     2 | 0 days 00:00:00 |    2.8  |
|     3 | 0 days 00:00:00 |    5.6  |

3. To homogenize the time measure with the traffic table we join the date with the time. The new date shape is ```date-month-year hour:minute:second```

4. Finally, we convert the different types of measurement into columns so that it looks more organized:

|      | date                |   weather_station |   direction |   humidity |   pressure |   rainfall |   solar |   temperature |   wind |
|-----:|:--------------------|------------------:|------------:|-----------:|-----------:|-----------:|--------:|--------------:|-------:|
|    0 | 2022-12-01 00:00:00 |                 4 |           0 |          0 |          0 |        0   |       0 |           7.1 |   0    |
|    1 | 2022-12-01 00:00:00 |                 8 |           0 |         67 |          0 |        0   |       0 |           9.4 |   0    |
|    2 | 2022-12-01 00:00:00 |                16 |           0 |         73 |          0 |        0   |       0 |           8.9 |   0    |

#### Traffic

Regarding the traffic table, the steps followed were the following:

1. First we filter out the values with measurement errors. In the documentation they tell us that these values are represented with an "N". 

2. We only keep the load measurement, which is what we are interested in for the analysis.

3. We group the values for each date and station and calculate the average.

#### Location

In order to combine the two previous tables, it's necessary to establish a correspondence between the traffic sensors and the weather stations. This can be achieved by employing a distance matrix and identifying the nearest pairs among the two types of station.  While direct measurement of this distance using coordinates is possible, we opted for the [Haversine distance](https://en.wikipedia.org/wiki/Haversine_formula) which calculates the distance in meters between a pair of coordinates, offering a clearer representation of distances.  This can be observed in the following heatmap, where the distances range from 0 to 20 km:

<figure>
<img src="image_source/heatmap3.png">
<figcaption style = "text-align: center"> <b>Figure 1:</b> Heatmap of distances in km between weather and traffic stations </figcaption>
</figure>
    
#### The Final Table <a class="anchor" id="t23"></a>

After preparing the three tables - weather, traffic, and distance - we merge them. Since the weather table has time intervals of 1 hour, whereas the traffic data is in 15-minute intervals, we will combine these tables using an 'asof' join. Ultimately, we can incorporate time and day-of-the-week information to analyze its relationship with traffic congestion.

|    | date                |   traffic_station |   load |   Distance |   Closest |   weather_station |   direction |   humidity |   pressure |   rainfall |   solar |   temperature |   wind |   weekday |   hour |
|---:|:--------------------|------------------:|-------:|-----------:|----------:|------------------:|------------:|-----------:|-----------:|-----------:|--------:|--------------:|-------:|----------:|-------:|
|  0 | 2022-12-01 00:00:00 |              1001 |      0 |         15 |         1 |                 8 |           0 |         67 |          0 |          0 |       0 |           9.4 |      0 |         3 |      0 |
|  1 | 2022-12-01 00:00:00 |              1002 |      0 |         15 |         1 |                 8 |           0 |         67 |          0 |          0 |       0 |           9.4 |      0 |         3 |      0 |
|  2 | 2022-12-01 00:00:00 |              1003 |      0 |         14 |         1 |                 8 |           0 |         67 |          0 |          0 |       0 |           9.4 |      0 |         3 |      0 |
|  3 | 2022-12-01 00:00:00 |              1006 |      0 |         15 |         1 |                 8 |           0 |         67 |          0 |          0 |       0 |           9.4 |      0 |         3 |      0 |
|  4 | 2022-12-01 00:00:00 |              1009 |      0 |         14 |         0 |                 4 |           0 |          0 |          0 |          0 |       0 |           7.1 |      0 |         3 |      0 |
    
    

### Data interpretation <a class="anchor" id="t24"></a>

Traffic is highly time-dependent. The data will be filtered to extract the peak moments where traffic typically is heavier. This will enhance our understanding of the data. Weekdays tend to have more traffic than holidays, just as daytime have higher traffic flow than nighttime. In the following figure underscores the evident seasonality within the dataset:

<table>
    <tr>
    <td style='text-align:center;'>
<figure>
<img src="image_source/loadperhour.png">
<figcaption style = "text-align: center">   <b>Figure 2:</b> Load per Hour </figcaption>
</figure>
 </td>
    <td>
<figure>
<img src="image_source/loadperweekday.png">
<figcaption style = "text-align: center">   <b>Figure 3:</b> Load per Weekday </figcaption>
</figure>
            </td>
    </tr>
</table>

Regarding the Rainfall-Load dependency we started seeing that rainy days are very few on our dataset:

|       |       rainfall |
|:------|---------------:|
| count | 841068         |
| mean  |      0.0130932 |
| std   |      0.194232  |
| min   |      0         |
| 50%   |      0         |
| 90%   |      0         |
| 99.9% |      2.9       |
| max   |     10.9       |

Examining the percentiles of the precipitation column, it becomes evident that there are very few recordings with rain. To address this, measurements were categorized into distinct classes based on the intensity of rain. Consequently, a separate analysis was conducted for data involving heavy rain, moderate to light rain, and no rain. The analysis was done hourly to mitigate the temporal dependence of traffic. The average increase in traffic congestion hours due to rain ranges from 5% to 14%, consistent with the studies presented in [the Use Case](#t12) chapter.

<figure>
<img src="image_source/rainfall.png">
<figcaption style = "text-align: center"> <b>Figure 4:</b> The average traffic load per hour for the measurements with heavy rain (blue), moderate rain (brown) and no rain (green).</figcaption>
</figure>

To verify that these differences between groups are significant, we can perform an anova test. And we see that in all hours there is great evidence that the load is different between the different levels of rain. For example, for hour 12:

|             |          sum_sq |    df |        F |       PR(>F) |
|:------------|----------------:|------:|---------:|-------------:|
| C(rainfall) | 37650.7         |     6 |  20.1144 |   1.3346e-23 |
| Residual    |     8.01363e+06 | 25687 | nan      | nan          |

It was decided to include rainfall within the model. The rest of the meteorological conditions did not give such a clear result, so they were left out.

### The Model <a class="anchor" id="t25"></a>

To assess performance, we created a toy model using a simple LSTM with 5-step memory to predict the load at a specific traffic station. The input includes the precedings data of the load, the rainfall, the hour and day of the week. The results for a single station seem quite positive as can be seen in [Graph 6](#Image61). 

(TBD: explain the next figures a little bit: axis might be enough)

<table> <a class="anchor" id="Image61"></a>
    <tr>
    <td style='text-align:center;'>
<figure>
<img src="image_source/loss_graph_p.png">
<figcaption style = "text-align: center">   <b>Figure 5:</b> Train vs Test loss plot  </figcaption>
</figure>
 </td>
    <td>
<figure>
<img src="image_source/fc_p.png">
<figcaption style = "text-align: center">   <b>Figure 6:</b> Traffic Forecasting for a traffic station </figcaption>
</figure>
            </td>
    </tr>
</table>


## PyKX migration <a class="anchor" id="t3"></a>

While the forecasting results obtained from the LSTM model were promising, there were certain considerations related to the efficiency of the Pythonic code used for project implementation. Firstly, the preprocessing of the datasets required an excessive amount of time. Specifically, the average time for loading and processing a month's worth of traffic information was 43.9 seconds. Similarly, the computational expenses associated with running the LSTM model for an individual station and a single month's data proved to be unfeasible within the project budget. All of these factors contributed to the team's concerns about the slowness and its potential impact on real-time data processing. Enter PyKX to the rescue! This section will detail how PyKX aided us in offloading heavy processing to kdb+/q. 



<div class="alert alert-block alert-info">
    <b> 🔍 You can find on Github the <a href="https://github.com/hablapps/AllRoadsLeadToPyKX/blob/Python-Version-Pre/AllRoadsLeadToPyKX.md">Original Python Project</a> that will be migrated below into PyKX. </b>
</div>



First of all we need to install and import PyKX:

In [ ]:
!pip install pykx

In [1]:
import pykx as kx

<div class="alert alert-block alert-info">
<b> 🔍 A license is required to use some of the following features. You can find more information in <a href="https://code.kx.com/pykx/1.6/getting-started/installing.html">
    PyKX installation documentation</a>. </b>
</div> 

## Datasets <a class="anchor" id="t31"></a>

Before continuying, be sure to download the [datasets](#t21) required for running the next cells.

#### Weather

The loading of the data will be done with the utilities provided by [PyKX](https://code.kx.com/pykx/1.4/api/read.html): (TBD: Why? It could be read as well with python)

In [52]:
weather = kx.q.read.csv('../dic_meteo22.csv', types='I'*4 + '*'*4 + 'FS'*24, delimiter=';')

<div class="alert alert-block alert-success" style="background-color: #FFFFFF; border: 0px solid; padding: 0px;box-shadow: 2px 2px #3841459c; border-left: 5px solid #00b100;margin-left: 5%; margin-right: 5%;">
<div class="alert-header custom-card-header" style="background-color: #d8ffd8; border: 1px solid #d8ffd8; color: black; padding: 10px; font-size: 115%;">
<i class="fa fa-commenting" aria-hidden="true" style="color:#00b100"></i>&nbsp;&nbsp;&nbsp;MORE INFO: <a href="https://code.kx.com/pykx/1.6/api/read.html">PyKX Read/Write Utils </a>
</div>
<div style="background-color: #e5ffdf70;  padding: 40px; padding-top:30px; padding-bottom:20px">
<p style="color: black; margin-top:0%; text-align: left;margin-left: 2%; margin-right: 5%; margin-bottom: 15px;font-size: 115%; line-weight: 1.5"> PyKX offers several functions that help loading and writing data. These functions allow you to use very different files types, both general and q-specific. The latter allows you to move tables from the q context to the Python context and vice versa.
</p>
</div>
</div>

The parameters of the `read.csv` function are quite standard. The URL to the file and the delimiter are indicated. We highlight the *types* parameter that expects the q [types](https://code.kx.com/q/basics/datatypes/) of each column. Our table is now a PyKX object:

In [53]:
type(weather)

pykx.wrappers.Table

Let's look at the first few rows of this table:

In [54]:
print(weather[:3])

PROVINCIA MUNICIPIO ESTACION MAGNITUD PUNTO_MUESTREO   ANO    MES  DIA  H01  ..
-----------------------------------------------------------------------------..
28        79        108      81       "28079108_81_98" "2022" "12" "22" 270  ..
28        79        108      82       "28079108_82_98" "2022" "12" "22" 9    ..
28        79        108      83       "28079108_83_98" "2022" "12" "22" 94.7 ..


<div class="alert alert-block alert-success" style="background-color: #FFFFFF; border: 0px solid; padding: 0px;box-shadow: 2px 2px #3841459c; border-left: 5px solid #00b100;margin-left: 5%; margin-right: 5%;">
<div class="alert-header custom-card-header" style="background-color: #d8ffd8; border: 1px solid #d8ffd8; color: black; padding: 10px; font-size: 115%;">
<i class="fa fa-commenting" aria-hidden="true" style="color:#00b100"></i>&nbsp;&nbsp;&nbsp;MORE INFO: <a href="https://code.kx.com/pykx/1.6/user-guide/fundamentals/indexing.html">Use pythonic indexing </a>
</div>
<div style="background-color: #e5ffdf70;  padding: 40px; padding-top:30px; padding-bottom:20px">
<p style="color: black; margin-top:0%; text-align: left;margin-left: 2%; margin-right: 5%; margin-bottom: 15px;font-size: 115%;line-weight: 1.5"> Accessing data from PyKX objects such as lists or tables can be done similarly to Numpy or Pandas. It allows us to index PyKX objects without the need to explicitly use q functions. And we can even index by columns! </p>        
<code style="background-color: #eee;border: 1px solid #999;display: block;padding: 10px;margin-left: 5%;margin-right: 10%;margin-bottom: 0px; color:black"> >>> print(weather["H01"][:3]) 
<div style="color:#808080;display:inline;"> 270 9 94.7 </div></code><br>   
</div>
</div>

We can pass the table to Pandas to see it in the markdown format:

In [55]:
weather[:3].pd()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,108,81,b'28079108_81_98',b'2022',b'12',b'22',270.0,N,...,218.0,V,228.0,V,227.83,V,213.67,V,233.83,V
1,28,79,108,82,b'28079108_82_98',b'2022',b'12',b'22',9.0,N,...,10.0,V,10.0,V,9.00,V,9.00,V,8.00,V
2,28,79,108,83,b'28079108_83_98',b'2022',b'12',b'22',94.7,N,...,86.7,V,91.4,V,93.80,V,96.30,V,98.70,V


<div class="alert alert-block alert-success" style="background-color: #FFFFFF; border: 0px solid; padding: 0px;box-shadow: 2px 2px #3841459c; border-left: 5px solid #00b100;margin-left: 5%; margin-right: 5%;">
<div class="alert-header custom-card-header" style="background-color: #d8ffd8; border: 1px solid #d8ffd8; color: black; padding: 10px; font-size: 115%;">
<i class="fa fa-commenting" aria-hidden="true" style="color:#00b100"></i>&nbsp;&nbsp;&nbsp;MORE INFO: Use <a href="https://code.kx.com/pykx/1.6/user-guide/fundamentals/creating.html">Python/Q Transform</a> 
</div>
<div style="background-color: #e5ffdf70;  padding: 40px; padding-top:30px; padding-bottom:20px">
<p style="color: black; margin-top:0%; text-align: left;margin-left: 2%; margin-right: 5%; margin-bottom: 15px;font-size: 115%; line-weight: 1.5"> Objects from q can be converted to pandas with <code>.pd()</code>, to PyArrow with <code>.pa()</code>, to numpy with <code>.np()</code> and to Python with <code>.py()</code> methods. And then back to q so that colleagues can continue the work with PyKX objects with <code>toq(pyObject)</code> function. This is great for Python developers new to PyKX because it allows them to take advantage of kdb+ databases while learning q.</p>        
</div>
</div>

As we can see, it's not inside the q memory space. Let's see how to access these objects and how to use q features on them. Let's start with the simple `xcol` function, which allows us to rename columns:

In [56]:
weather = kx.q.xcol({'ANO': 'year', 'MES': 'month', 'DIA': 'day', 'ESTACION':'weather_station', 'MAGNITUD':'magnitude'}, weather)

<div class="alert alert-block alert-success" style="background-color: #FFFFFF; border: 0px solid; padding: 0px;box-shadow: 2px 2px #3841459c; border-left: 5px solid #00b100;margin-left: 5%; margin-right: 5%;">
<div class="alert-header custom-card-header" style="background-color: #d8ffd8; border: 1px solid #d8ffd8; color: black; padding: 10px; font-size: 115%;">
<i class="fa fa-commenting" aria-hidden="true" style="color:#00b100"></i>&nbsp;&nbsp;&nbsp;MORE INFO: <a href="https://code.kx.com/pykx/1.6/user-guide/fundamentals/evaluating.html">Use q functions </a>
</div>
<div style="background-color: #e5ffdf70;  padding: 40px; padding-top:30px; padding-bottom:20px">
<p style="color: black; margin-top:0%; text-align: left;margin-left: 2%; margin-right: 5%; margin-bottom: 15px;font-size: 115%; line-weight: 1.5"> Many q functions are available through the <code>pykx.q</code> interface. The attributes are identical to those expected in the q function but using Python structures instead. These functions are compiled and perfectly explained in the <a href="https://code.kx.com/pykx/1.6/api/q/q.html"> PyKX documentation</a></p>
<code style="background-color: #eee;border: 1px solid #999;display: block;padding: 10px;margin-left: 5%;margin-right: 10%; color:black"> >>> kx.q.distinct(['A', 'B', 'B', 'B' ,'C'])
<div style="color:#808080;display:inline;"> `A`B`C </div></code><br> 
<p style="color: black; margin-top:0%; text-align: left;margin-left: 2%; margin-right: 5%; margin-bottom: 15px;font-size: 115%; line-weight: 1.5"> For those who want to dig deeper into kdb+/q and gain experience, you can use q functions and pass PyKX objects as arguments: </p>
<code style="background-color: #eee;border: 1px solid #999;display: block;padding: 10px;margin-left: 5%;margin-right: 10%; color:black"> >>> kx.q("distinct", ['A', 'B', 'B', 'B' ,'C'])
<div style="color:#808080;display:inline;"> `A`B`C </div></code><br> 
<p style="color: black; margin-top:0%; text-align: left;margin-left: 2%; margin-right: 5%; margin-bottom: 15px;font-size: 115%; line-weight: 1.5"> Note that in the previous function a Python object is being passed to a q function. If Python objects have a direct equivalent in q, such as dictionaries, they can be directly used as the attributes for PyKX functions. Underneath, PyKX takes care of transforming it to q. </p>
<p style="color: black; margin-top:0%; text-align: left;margin-left: 2%; margin-right: 5%; margin-bottom: 15px;font-size: 115%; line-weight: 1.5"> You can also apply <a href="https://code.kx.com/q/basics/iteration/">q iterations</a> to functions: </p>
<code style="background-color: #eee;border: 1px solid #999;display: block;padding: 10px;margin-left: 5%;margin-right: 10%; color:black"> >>> kx.q("lower").each(['A', 'B', 'C'])
<div style="color:#808080;display:inline;"> `a`b`c </div></code><br>   
<p style="color: black; margin-top:0%; text-align: left;margin-left: 2%; margin-right: 5%; margin-bottom: 15px;font-size: 115%; line-weight: 1.5"> Finally, you can even create functions and use them with PyKX or Python objects.</p>
<code style="background-color: #eee;border: 1px solid #999;display: block;padding: 10px;margin-left: 5%;margin-right: 10%; color:black"> >>> kx.q("{u !(sum x=) each u:distinct x}", ['A', 'B', 'B', 'B' ,'C'])
<div style="color:#808080;display:inline;"> A| 1
 B| 3
 C| 1 </div></code><br>   
</div>
</div>

Apart from this functions we have the `kx.q.qsql` interface, which allows us to query into tables. Specifically we can use `select`, `exec`, `update` and `delete`, which share some common characteristics, mainly with the arguments they receive.

The first three share roughly this function call structure:

`kx.q.qsql.{function}({tab}, columns=..., where=..., by=...)`

The `columns` argument expects either a list of strings or a dictionary where the key is the column name and the value is the actual value you want in this column if let's say you want to apply a function to it. Let's look at this specific example:

In [57]:
weather = kx.q.qsql.delete(weather, ['PUNTO_MUESTREO', 'PROVINCIA', 'MUNICIPIO'])

With the above code we have removed several columns that are not relevant to the analysis.

<div class="alert alert-block alert-success" style="background-color: #FFFFFF; border: 0px solid; padding: 0px;box-shadow: 2px 2px #3841459c; border-left: 5px solid #00b100;margin-left: 5%; margin-right: 5%;">
<div class="alert-header custom-card-header" style="background-color: #d8ffd8; border: 1px solid #d8ffd8; color: black; padding: 10px; font-size: 115%;">
<i class="fa fa-commenting" aria-hidden="true" style="color:#00b100"></i>&nbsp;&nbsp;&nbsp;MORE INFO: <a href="https://code.kx.com/pykx/1.6/api/read.html">qSQL</a> and  <a href="https://code.kx.com/pykx/1.6/api/read.html">SQL</a> querys
</div>
<div style="background-color: #e5ffdf70;  padding: 40px; padding-top:30px; padding-bottom:20px">
<p style="color: black; margin-top:0%; text-align: left;margin-left: 2%; margin-right: 5%; margin-bottom: 15px;font-size: 115%; line-weight: 1.5"> qSQL queries are very powerful and concise at the same time. As we have seen PyKX allows you to use qSQL queries using API functions.</p>
<code style="background-color: #eee;border: 1px solid #999;display: block;padding: 10px;margin-left: 5%;margin-right: 10%; color:black"> >>> print(kx.q.qsql.select(weather, {"magnitude": "count distinct magnitude"}, by=["weather_station"])[:3])
<br><div style="color:#808080;display:inline;"> weather_station| magnitude
 ---------------| ---------
 4              | 1        
 8              | 2        
 16             | 2     
</div>         
</code>
<br>
<p style="color: black; margin-top:0%; text-align: left;margin-left: 2%; margin-right: 5%; margin-bottom: 15px;font-size: 115%; line-weight: 1.5"> But if you are more familiar with the q environment it is also possible to use q functions. This reduces the verbosity of python functions.
</p>
<code style="background-color: #eee;border: 1px solid #999;display: block;padding: 10px;margin-left: 5%;margin-right: 10%; color:black"> >>> print(kx.q("{select count distinct magnitude by weather_station from x}", weather)[:3])
<br> <div style="color:#808080;display:inline;">weather_station| magnitude
 ---------------| ---------
 4              | 1        
 8              | 2        
 16             | 2 
</div>    
</code>
<br>
<p style="color: black; margin-top:0%; text-align: left;margin-left: 2%; margin-right: 5%; margin-bottom: 15px;font-size: 115%; line-weight: 1.5"> PyKX lets you use SQL queries too! </p>
<code style="background-color: #eee;border: 1px solid #999;display: block;padding: 10px;margin-left: 5%;margin-right: 10%; color:black"> >>> print(kx.q.sql("SELECT weather_station, COUNT(DISTINCT(magnitude)) FROM &#36 1 GROUP BY weather_station", weather)[:3])
<br> <div style="color:#808080;display:inline;">weather_station magnitude
 -------------------------
 4               1        
 8               2        
 16              2             
</div></code>
<br>
</div>
</div>

The next task we need to do is combine the year, month, and day into a single date column. To do this, we started by accessing these three columns. This can be done by indexing:

In [58]:
print(weather["year", "month", "day"])

"2022" "2022" "2022" "2022" "2022" "2022" "2022" "2022" "2022" "2022" "2022" ..
"12"   "12"   "12"   "12"   "12"   "12"   "12"   "12"   "12"   "12"   "12"   ..
"22"   "22"   "22"   "22"   "22"   "22"   "22"   "01"   "02"   "03"   "04"   ..


We see that the result is three lists of the sample size. Our goal is a list of sample size with the three elements that make up the date joined together:

In [59]:
print(kx.q.flip(weather["year", "month", "day"])[:3])

"2022" "12" "22"
"2022" "12" "22"
"2022" "12" "22"


Looks like we're getting clos. Now we have a sample size list with a list in each position containing 3 elements: the day, the month and the year. To join each set of the list into a single joined element, the *each* iterator can be used:

In [60]:
print(kx.q.each(kx.q.raze, kx.q.flip(weather["year", "month", "day"]))[:3])

"20221222"
"20221222"
"20221222"


All that remains to do is to convert it from string to date. Unfortunately, some functions (especially the overloaded glyphs) are not yet implemented. For example cast (`$`), take (`#`), concat (`,`)... So we are forced to abandon pythonic way of calling q functions and perform this casting writing kdb+/q code using the pykx.q method:

In [61]:
date = kx.q('"D"$',(kx.q.each(kx.q.raze, kx.q.flip(weather["year", "month", "day"]))))
print(date[:3])

2022.12.22 2022.12.22 2022.12.22


Finally, we add this column to our table:

In [62]:
weather_ = kx.q.qsql.update(weather, columns = {'date': date})

Some team members started using q code instead of PyKX functions in the pythonic way, as they found the resulting code to be more elegant and concise. Once the team had a little more fluency, a function written entirely in q was proposed:

In [63]:
weather = kx.q.qsql.update(weather, columns = {'date':'"D"$ raze each flip(year;month;day)'})

Again, both solutions are fully compatible, even combinable. It is up to the programmer to use one methodology or the other. 

The three columns that are already included in date can now be removed:

In [64]:
weather = kx.q.qsql.delete(weather, ['year', 'month', 'day'])

The current state of the weather table is as follows:

In [65]:
weather[:3].pd()

,weather_station,magnitude,H01,V01,H02,V02,H03,V03,H04,V04,...,V20,H21,V21,H22,V22,H23,V23,H24,V24,date
0,108,81,270.0,N,252.0,N,216.0,N,242.0,N,...,V,228.0,V,227.83,V,213.67,V,233.83,V,2022-12-22
1,108,82,9.0,N,8.0,N,9.0,N,8.0,N,...,V,10.0,V,9.00,V,9.00,V,8.00,V,2022-12-22
2,108,83,94.7,N,97.6,N,96.6,N,97.5,N,...,V,91.4,V,93.80,V,96.30,V,98.70,V,2022-12-22


Now it's time to turn our attention to breaking down the H* and V* queries into multiple rows, and supplying a time column to avoid missing information. The way to proceed in q would be to use functional qSQL to select the columns that follow the previous patterns, but we are going to take advantage of the fact that the q code in PyKX is introduced through strings to avoid it:

In [66]:
def functionalSearch(cols, pattern, func):
    xcols = cols[kx.q.where(kx.q.like(cols, pattern))]
    xstring = func.format(kx.q.sv(b";", kx.q.string(xcols)).py().decode("utf-8"))
    return xcols, xstring

The above function receives a list of columns, a pattern to search for, and a q function in string format that passes as an argument the columns found following that pattern in qSQL format (where columns are accessed with their names and not with a symbol). Applying this to all columns starting with *"H"* returns these columns as a vector of symbols and a string of these columns in qSQL format:

In [67]:
cols = kx.q.cols(weather)
found_columns, qsql_function = functionalSearch(cols, b'H*', "{}")

print("Columns found: ", found_columns)
print("qSQL function: ", qsql_function)

Columns found:  `H01`H02`H03`H04`H05`H06`H07`H08`H09`H10`H11`H12`H13`H14`H15`H16`H17`H18`H19`..
qSQL function:  H01;H02;H03;H04;H05;H06;H07;H08;H09;H10;H11;H12;H13;H14;H15;H16;H17;H18;H19;H20;H21;H22;H23;H24


This is very powerful, as it allows us to use qSQL with variables without having to use functional forms (usually complicated for first-time kdb/q developers).

Let's use the above for the columns beginning with **H**, which will give us the measurement value; and the columns that begin with **V**, which tell us if the measurement is valid or not. The function to convert the measurements column to rows is *flip*:

In [68]:
hcols, value = functionalSearch(cols, b'H*', "flip({})")
vcols, valid = functionalSearch(cols, b'V*', "flip({})")

Now we just have to pass our built-in functions in string format to the qSQL "update" function, together with the 24 hours repeated the initial number of rows:

In [69]:
weather = kx.q.qsql.update(weather, columns = {'hour': 'count[i]#enlist 01:00*til 24', 'values': value, 'valid': valid})

Columns starting with **H** or **V** can be removed by using the same trick we used before to prevent functional qSQL:

In [70]:
weather = kx.q.qsql.delete(weather, columns = kx.q.raze(hcols,vcols).py())

Let's see the result:

In [71]:
weather[:3].pd()

,weather_station,magnitude,date,hour,values,valid
0,108,81,2022-12-22,"[0 minutes, 60 minutes, 120 minutes, 180 minut...","[270.0, 252.0, 216.0, 242.0, 239.0, 246.0, 233...","[N, N, N, N, N, N, N, N, N, N, V, V, V, V, V, ..."
1,108,82,2022-12-22,"[0 minutes, 60 minutes, 120 minutes, 180 minut...","[9.0, 8.0, 9.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","[N, N, N, N, N, N, N, N, N, N, V, V, V, V, V, ..."
2,108,83,2022-12-22,"[0 minutes, 60 minutes, 120 minutes, 180 minut...","[94.7, 97.6, 96.6, 97.5, 97.5, 98.2, 98.8, 98....","[N, N, N, N, N, N, N, N, N, N, V, V, V, V, V, ..."


Now all we have to do is expand the table so that each element of the lists corresponds to a row:

In [72]:
weather = kx.q.ungroup(weather)

We can shrink the table a bit more by removing the rows that are not valid and joining the date with the time:

In [73]:
weather = kx.q.qsql.select(weather, where = 'valid=`V')
weather = kx.q.qsql.update(weather, columns = {'date': 'date+hour'})
weather = kx.q.qsql.delete(weather, columns = ["valid", "hour"])
weather[:3].pd()

,weather_station,magnitude,date,values
0,108,81,2022-12-22 10:00:00,263.00
1,108,81,2022-12-22 11:00:00,247.83
2,108,81,2022-12-22 12:00:00,215.83


<div class="alert alert-block alert-success" style="background-color: #FFFFFF; border: 0px solid; padding: 0px;box-shadow: 2px 2px #3841459c; border-left: 5px solid #00b100;margin-left: 5%; margin-right: 5%;">
<div class="alert-header custom-card-header" style="background-color: #d8ffd8; border: 1px solid #d8ffd8; color: black; padding: 10px; font-size: 115%;">
<i class="fa fa-commenting" aria-hidden="true" style="color:#00b100"></i>&nbsp;&nbsp;&nbsp;ALTERNATIVE FEATURE: Use <a href="https://code.kx.com/pykx/1.6/user-guide/advanced/numpy.html">Numpy</a> and  <a href="https://code.kx.com/pykx/1.6/user-guide/advanced/Pandas_API.html">Pandas</a>
</div>
<div style="background-color: #e5ffdf70;  padding: 40px; padding-top:30px; padding-bottom:20px">
<p style="color: black; margin-top:0%; text-align: left;margin-left: 2%; margin-right: 5%; margin-bottom: 15px;font-size: 115%;line-weight: 1.5"> Those of us who are not yet so familiar with the kdb+/q ecosystem, we can continue using part of Numpy's functionality. Specifically   <a href="https://numpy.org/doc/stable/reference/ufuncs.html">universal functions</a>. Using this type of function, the mean of an array can be coded as:</p> 
<code style="background-color: #eee;border: 1px solid #999;display: block;padding: 10px;margin-left: 5%;margin-right: 10%;color:black"> >>> import numpy as np
 >>>
 >>> def numpy_mean(arr):
 >>>   return np.add.reduce(arr) / len(arr)
 >>>
 >>> print(numpy_mean(kx.toq([1,4,6,7,9])))    
<div style="color:#808080;display:inline;">5.4 </div></code><br>
<p style="color: black; margin-top:0%; text-align: left;margin-left: 2%; margin-right: 5%; margin-bottom: 15px;font-size: 115%; line-weight: 1.5"> Pandas can be used with PyKX objects using the Pandas API. It can be used simply by importing Numpy and Pandas and activating a flag. The Pandas API is still in the development phase, so it does not have all the functions of Pandas implemented yet. </p>     
<code style="background-color: #eee;border: 1px solid #999;display: block;padding: 10px;margin-left: 5%;margin-right: 10%;color:black"> >>> import os
 >>> os.environ['PYKX_ENABLE_PANDAS_API'] = 'true'
 >>> import numpy as np
 >>> import pandas as pd
 >>>
 >>> print(weather.iloc[weather["magnitude"] == "temperature", ["magnitude", "values"]][:5])        
<br> <div style="color:#808080;display:inline;"> magnitude   | values
  ------------| ---------
  temperature | 7.1         
  temperature | 6.6        
  temperature | 6.0    
</div></code>
<br>
</div>
</div>

The **magnitude** column is required to give semantics to the **value**. As established by the dataset creators, the different magnitudes correspond to the items that we collect in the following dictionary:

In [74]:
magnitude = {80:"ultraviolet", 
             81:"wind", 
             82:"direction", 
             83:"temperature", 
             86:"humidity", 
             87:"pressure", 
             88:"solar", 
             89:"rainfall"}

We just have to change the key by the value of the dictionary.

In [75]:
weather = kx.q('{update magnitude: x magnitude from y}', magnitude, weather)

Finally, all you have to do is separate the different weather conditions into different columns:

In [76]:
weather = kx.q('{exec (value x)#magnitude!values by date,weather_station from y}',magnitude,weather)

In [78]:
# (TBD: show some sample records of the table after each major transformation stage. For instance, here. But weather[:3].pd() doesn't work)

The exec query allows us to convert a dictionary stored in a column into multiple columns with the key as the column name and the values as the data in that column. This is useful when we create a dictionary made up of the weather conditions of each row associated with their values. By applying it and grouping we have the weather conditions in different columns for each weather entry and weather station.

#### Traffic

This second table will also be loaded into the Python environment (TBD: more specifically, in the Q environment within Python?):

In [79]:
traffic = kx.q.read.csv('../12-2022.csv', types="IPSIIIISI", delimiter=';')

We are interested in knowing the average load by date and by season, removing measurement errors. The power of qSQL allows us to do this in a single query:

In [80]:
traffic = kx.q.qsql.select(traffic,
                         columns = {'traffic_load': 'avg carga'},
                         by = {"date":'fecha', "traffic_station": 'id'}, 
                         where = "error=`N")



(TBD: show samples of the traffic table)

#### Location

Both traffic and weather station tables will be loaded into the q memory space:

In [ ]:
kx.q["weather_station"] = kx.q.read.csv('../Estaciones_control_datos_meteorologicos.csv', types=" IFF", delimiter=";", as_table=True)
kx.q["traffic_station"] = kx.q.read.csv('../pmed_ubicacion_04-2023.csv', types = "SII**FFFF", delimiter = ";", as_table=True)

We can now access these objects within q functions without needing to pass them as PyKX or Python objects. For example, let's change the name of the columns in both tables to standardize them:

In [ ]:
kx.q("weather_station:(`CODIGO_CORTO`LONGITUD`LATITUD!`weather_station`longitude`latitude)xcol weather_station")
_=kx.q("traffic_station:(`id`longitud`latitud!`traffic_station`longitude`latitude)xcol traffic_station")

<div class="alert alert-block alert-success" style="background-color: #FFFFFF; border: 0px solid; padding: 0px;box-shadow: 2px 2px #3841459c; border-left: 5px solid #00b100;border-top: 2px solid #00b100;margin-left: 5%; margin-right: 5%;">
<div class="alert-header custom-card-header" style="background-color: #d8ffd8; border: 1px solid #d8ffd8; color: black; padding: 10px; font-size: 115%;">
<i class="fa fa-commenting" aria-hidden="true" style="color:#00b100"></i>&nbsp;&nbsp;&nbsp;MORE INFO: Use <a href="https://code.kx.com/pykx/1.6/user-guide/fundamentals/creating.html#by-retrieving-a-named-entity-from-qs-memory">Q memory space</a> 
</div>
<div style="background-color: #e5ffdf70;  padding: 40px; padding-top:30px; padding-bottom:20px">
<p style="color: black; margin-top:0%; text-align: left;margin-left: 2%; margin-right: 5%; margin-bottom: 15px;font-size: 115%; line-weight: 1.5"> If you feel more comfortable programming in q, you can work in the q memory space. PyKX objects can be passed into the q memory space to work with them as if you were in a q ecosystem like so: <code>kx.q["table"]</code>. Then to bring them back to the Python memory space, you just have to return them using q code: <code>kx.q("table")</code></p>       
</div>
</div>

(TBD: border-top: 2px solid #00b100)

Our goal is to join these two tables. There seems to be no identifier that allows us to do a standard join. However, both the weather and traffic stations are located by coordinates. We can use the distance between the stations for measuring traffic and weather to join them. To calculate the distance between two coordinates, the Harvesine distance can be used as discussed earlier. This distance function is already developed in Python but it is not available on q.

One option would be to reimplement it in q, but this would not be feasible in case we were dealing with more complex libraries. Although slower, we could pass our q objects to Python and work with them. However, it is advisable to keep using q objects for as long as possible. These features we just explained for moving from Python objects to q and vice versa allow us, at least temporarily, to reuse Python code. It is also reasonable for the size of tables we are working with in this step. To introduce our q objects in this function, we can use some of the PyKX tools to transform them into Python objects that we talked about in [Learning Pykx](). (TBD: Learning PyKx no longer available)

In [ ]:
pip install haversine

In [ ]:
from haversine import haversine_vector, Unit
dist = kx.toq(
            haversine_vector(kx.q('`longitude`latitude # weather_station').pd(), 
                             kx.q('`longitude`latitude # traffic_station').pd(),
                             Unit.KILOMETERS, comb=True))

We now have a matrix that measures the distance in kilometers for each pair of traffic and weather stations. We can select the pairs of station identifiers whose minimum distance:

In [ ]:
ids = kx.q.each(kx.q('{first where x=min x}'), dist)
distance_table = kx.q('{traffic_station ^ weather_station[x]}' ,  ids)
distance_table = kx.q.qsql.delete(distance_table, columns = ['tipo_elem','distrito','cod_cent','nombre','utm_x','utm_y','longitude', 'latitude'])

#### Final Table

The joining of the three tables is fairly straightforward. The distances table can be joined with any of the other two by means of a simple left join. The traffic and weather ones have to be joined with an asof join as they have different time intervals. Finally, two columns need to be added to convey the seasonality of the data to the model: time and day of the week.

In [ ]:
complete = kx.q.lj(traffic, kx.q.xkey('traffic_station', distance_table))
complete = kx.q.aj(kx.toq(['weather_station', 'date']), complete, weather)
complete = kx.q.qsql.update(kx.q("0^",complete),  {"hour":"`hh$date", "weekday":'("d"$date)mod 7'})

Let's look at this last table:

In [ ]:
kx.q("5#",complete).pd()

### Model <a class="anchor" id="t32"></a>

For the model's ingestion, we opt for the selection of only the necessary columns. Additionally, we normalize the rainfall column using a simple MinMax scaler. This function can be included within the q memory spacefor whenever it's required.

In [ ]:
kx.q("minMaxScale:{[l] ({(x-y)%(z-y)}[;min l;max l]')l}")
                  
final = kx.q.qsql.select(complete, columns = {"date": "date",
                                              "traffic_station":"traffic_station",
                                              "hour":"hour", 
                                              "weekday": "weekday", 
                                              "traffic_load": "traffic_load%100", 
                                              "temperature":"minMaxScale temperature", 
                                              "rainfall":"minMaxScale rainfall"}
                                    )

During this migration from Pandas, the main hassle was to migrate the `time_window` function, as it relied on loops. The way we approached it was to first understand the input we had, the output we needed and then think of an idiomatic way to implement it using q instead of doing a 1:1 migration, which would have probably be more costly in terms of time. In this case, our input was a table and our output was a list of matrices for each station. We created several functions that helped us during the process:

* **sliding window**: given a matrix, x, divides it into chunks of length y:

In [ ]:
_=kx.q("""sw:{({y#z _x}[x;y;]')til count b:y _x}""")

* **get target**: For a given matrix, x, it takes the target located in the column z and the first lines are eliminated to match with the chunks length y:

In [ ]:
_=kx.q("""gt:{y _(flip x)[z]}""") # gets target (in position z)

* **to Matrix**: Transform the table x to a matrix:

In [ ]:
_=kx.q("""toMatrix:{({[t;i]value t[i]}[x;]')til count x:flip x}""") # / table to matrix

* **prepareData**: Create the train and test datasets. Select the model needed columns *columns* from the table grouped by traffic station. For each traffic station it lefts *ntest* rows for test. It divide the data with chunks of length *chunkLen*. Finally it create a list of dictionaries for training data and training target (the load column) and test data target.
    
    First off, regardless of whether we needed the train or test output, we needed to get either the last *ntest* records or up to the last *ntest* records. Since we needed to repeat this operation for the given columns, we decided that a functional statement would be the best fit.

    Once we shaped the data the way we needed, it was time to build the list of matrices. This needed to be done using a sliding window but, in order to return a list of matrices (not a list of tables), we used the **toMatrix** function to transform a table to a matrix and **sw** (sliding window) function to apply the sliding window based on a single huge matrix.

    As for the y data, we used the function **gt** (get target) that returned the target (traffic load) as a list with the first *chunkLen*+1 entries removed:

In [ ]:
_=kx.q("""
        prepareData:{[data; ntest; chunkLen; columns; locTarget]  
            train:(toMatrix')?[data;();`traffic_station;columns!({(y;(-;(count;x);z);x)}[;_;ntest]')columns]; 
            test:(toMatrix')?[data;();`traffic_station;columns!({(y;(-;(count;x);z);x)}[;#;ntest]')columns];                                                                               
            (((sw[;chunkLen]')test;(gt[;chunkLen;locTarget]')test);((sw[;chunkLen]')train;(gt[;chunkLen;locTarget]')train))   
        }
    """)

Lets test this function with only one station:

In [ ]:
import numpy as np

In [ ]:
station_id = 4010

station = kx.q.qsql.select(final, where=["traffic_station="+str(station_id)])

data = kx.q("prepareData", station, 500, 5, kx.SymbolVector(['rainfall', 'traffic_load', 'hour', 'weekday']), 1)

X_train, y_train = np.array(data[0][0][station_id].py()), np.array(data[0][1][station_id].py())
X_test, y_test =  np.array(data[1][0][station_id].py()), np.array(data[1][1][station_id].py())

And now we will try this datasets with a simple LSTM model:

In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
import numpy as np
import matplotlib.pyplot as plt


model = Sequential()

model.add(LSTM(units = 50, return_sequences=False, input_shape=[None,4]))
model.add(Dense(units = 1))

model.compile(loss='mae', optimizer='adam')

history = model.fit(X_train, y_train, 
                    epochs=50, batch_size=8, 
                    validation_data=(X_test, y_test), 
                    verbose=0, shuffle=False)

plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')

plt.legend()
plt.show()

We can see the performance of the model in the following graph:

In [ ]:
plt.plot(y_test, label='test real')
plt.plot(range(400,495), model.predict(X_test[400:], verbose=0).flatten(), label='test predict')
plt.legend()
plt.show()

(TBD: performance gains? which are the key aspects to be migrated into q? pandas queries ... anything else? What I'd like is something like: it's clear what we have to do: read and clean tables, etc. This is done in python using pandas, using ... whatever. What we are going to do is to do this part using qsql, this other using ... You have to think of the following situation: with all you know now, you are requested to tell someone how to migrate some pythonic code into pykx, what would you tell her to start from? which things are the most important one to migrate? etc.)


<div class="alert alert-block alert-info">
    <b> 🔍 You can find on Github the resulting <a href="https://github.com/hablapps/AllRoadsLeadToPyKX/blob/Python-Version-Pre/AllRoadsLeadToPyKX.md">PyKX-enhanced Pythonic version</a>. </b> (TBD: fix url)
</div>



## pykx.q migration <a class="anchor" id="t4"></a>

When the team got comfortable with the q language they started using it almost entirely through `kx.q()`, as it was more concise. But the programming became somewhat tedious having to use strings. This is why it was decided to jump to a q environment. But as seen in the previous chapter, there was Python code that the team couldn't port to q. For this reason it was decided to stay in PyKX but this time in the q environment.

**pykx.q** allows Python code execution on a q environment and, as a result, it opens up the door for new opportunities when dealing with existing codebases as it allows for importing and using Python libraries, both installed on the system and available as .py files.

In our case we use both of these options. We will focus on these opportunities to use Python code in pykx.q and leave the link to the project's [q script]() for the interested reader.

<div class="alert alert-block alert-info">
    <b> 🔍 You can find on Github the resulting <a href="https://github.com/hablapps/AllRoadsLeadToPyKX/blob/Python-Version-Pre/AllRoadsLeadToPyKX.md">PyKX-enhanced kdb+/q version</a>. (TBD: fix url)</b>
</div>

First, we want to execute the `haversine_vector` function to measure the distance between coordinates. Since it's not available on q, we decided to use pykx.q to incorporate this library straight into our q code with these lines:

```q
.pykx.pyexec"from haversine import haversine_vector, Unit";
```

This function expects two Pandas DataFrames as input, so we need to change the default conversion type from "np" or Numpy to "pd" or Pandas:

```q
.pykx.setdefault"pd";
```

Having done this, we can "move" our input variables to the Python memory space using `.pykx.set`

```q
.pykx.set[`a;`longitude`latitude#a];
.pykx.set[`b;`longitude`latitude#b];
```

And finally execute our function

```q
(.pykx.eval"haversine_vector(a, b, Unit.KILOMETERS, comb=True)")`
```

Notice the backtick at the end, this is for converting back to a q type.



The other way we can run Python code is to load a .py (renamed to .p) file using `\l`. This could be done as follows:

```q
system"l kerasmodel.p";
```

Here we have defined two functions named `fit` and `predict`. We can load them and use them like this:

```q
modelfit:.pykx.get`fit;
modelfit[train[0][3403];train[1][3403];test[0][3403];test[1][3403]];
modelpredict:.pykx.get`predict;
res:modelpredict[train[0][3403]];
```

(TBD) Some performance results from this version?

## Final thoughts <a class="anchor" id="t5"></a>

Ultimately, the project was a resounding success. We succeeded in transforming a codebase plagued by disorganization and performance concerns into a streamlined and competitive solution, empowered by Kdb+/q. PyKX is designed in a way that allows more experienced profiles and new kdb+/q developers to coexist in the same development environment. This favors the collaboration among a diversely experienced team. This is thanks to the different levels of integration that PyKX has and their compatibility with each other as it serves as an accessible introduction to Kdb+/q for less-experienced team members, further enhancing our team's capabilities.

The pykx.q library allows, as we have discussed earlier, to use existing Python codebases. This feature is really useful, specially when dealing with code migrations from Python. Through our experience, we identified challenges in aligning Python input shapes with their counterparts converted from q. After using the `.pykx.set` function for conversion to Python, we often needed to manipulate these transformed objects within the Python environment or modify the default conversion type to match the expected input format of Python functions.. Once that was dealt with, the experience using this library was really nice and technically impressive.

Since the data we needed to transfer back and forth between memory spaces was not that big (we were cautious this was the case) we observed no perceptible loss in performance. Instead, we achieved a significant improvement compared to Pandas.

Overall we would rate both PyKX libraries highly since it enables users to reuse existing code, significantly reducing the time required for migrations between these two languages.

## Bibliography <a class="anchor" id="t5"></a>

* Vidas M, Tubić V, Ivanović I, Subotić M. Sustainability (Basel) 2022; 14(9): e4985, http://dx.doi.org/10.3390/su14094985  
* Y. Yao et al., "Analyzing the Effects of Rainfall on Urban Traffic-Congestion Bottlenecks," in IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, vol. 13, pp. 504-512, 2020, http://dx.doi.org/10.1109/JSTARS.2020.2966591  
* Essien, A., Petrounias, I., Sampaio, P., & Sampaio, S. (2018). The Impact of Rainfall and Temperature on Peak
and Off-Peak Urban Traffic. In Database and Expert Systems Applications : 28th International Conference, DEXA
2018, Regensburg, Germany. (pp. 399-407). (Lecture Notes in Computer Science). Springer Nature.
https://doi.org/10.1007/978-3-319-98812-2_36  
(TBD: new refs - need formatting .. )
* A. Morrison and N. Crone, Accelerating Python Workflows using PyKX, June 2023, KX Developers blog, https://kx.com/blog/accelerating-python-workflows-using-pykx/
* Reuben Taylor. An Introduction to PyKXhttps://kx.com/videos/an-introduction-to-pykx/ June, 2023
* Paul Walsh PyKX Boosts Trade Analytics https://www.treliant.com/knowledge-center/pykx-boosts-trade-analytics/ June 26, 2023